# KRX API를 사용하여 모든 데이터 처리하여 나타내기

### 시작일은 23년 11월 12일부터 정확하게 1년 뒤인 24년 11월 12일로 설정함
### 보고 싶은 데이터의 기간을 변경할 경우 여기서 변경하면됨.

In [12]:
import requests
import pandas as pd
from datetime import datetime, timedelta

import zipfile
import xml.etree.ElementTree as ET
import os
import matplotlib.pyplot as plt
import seaborn as sns

import dart_fss as dart_fss


start_date = datetime(2023, 11, 12)
end_date = datetime(2024, 11, 12)

print('시작일자 : '+start_date.strftime("%Y%m%d"))
print('종료일자 : '+end_date.strftime("%Y%m%d"))

시작일자 : 20231112
종료일자 : 20241112


#### ─────────────────────────────────────────────────────────────────────────────────────────

## 코스피 지수를 kospi_df 에 저장하기
## 모든 요일별 데이터를 저장함.


In [13]:
#코스피 데이터 저장

# API URL
url = "http://data-dbg.krx.co.kr/svc/apis/idx/kospi_dd_trd" 

# 인증키 설정
auth_key = "956087495A1D4769A88B0F3411D0890EF02445EB"  # 제공받은 인증키로 대체하세요

# 헤더작성
headers = {
    "AUTH_KEY": auth_key,

}

kospi_df = pd.DataFrame()

current_date = start_date
while current_date <= end_date:
    bas_d = current_date.strftime("%Y%m%d")
    
    # 요청 파라미터 설정
    params = {
        "basDd": bas_d,
    }
    
    # API 요청
    response = requests.get(url, headers=headers, params=params)
    
    # 응답 확인 및 데이터 처리
    if response.status_code == 200:
        data = response.json()  # 응답 JSON 형식으로 파싱
        if "OutBlock_1" in data:
            df = pd.DataFrame(data["OutBlock_1"], columns=['BAS_DD', 'IDX_NM', 'CLSPRC_IDX', 'ACC_TRDVOL',
                                                           'ACC_TRDVAL', 'MKTCAP'])
            df2 = df[df['IDX_NM'] == '코스피']
            
            kospi_df = pd.concat([kospi_df, df2],ignore_index = True)
    else:
        print(f"Error {response.status_code}: {response.text}")
    
    # 다음 날짜로 이동
    current_date += timedelta(days=1)
    
print(kospi_df)

# 2024년 11월 12일자 코스피 지수 출력한다면?
print('2024년 11월 12일자 코스피 지수는? '+kospi_df[kospi_df['BAS_DD'] == '20241112']['CLSPRC_IDX'])

       BAS_DD IDX_NM CLSPRC_IDX ACC_TRDVOL      ACC_TRDVAL            MKTCAP
0    20231113    코스피    2403.76  306595374   5932781183278  1917613408659413
1    20231114    코스피    2433.25  306537170   6380927423786  1941186703678621
2    20231115    코스피    2486.67  419290344   9326776727601  1983027043761656
3    20231116    코스피    2488.18  406910631   6804642810555  1984471072736332
4    20231117    코스피    2469.85  388169286   8127375737844  1973334760360969
..        ...    ...        ...        ...             ...               ...
239  20241106    코스피    2563.51  547387428  11787017691104  2087643693499376
240  20241107    코스피    2564.63  464621422  10805202820569  2088431969039404
241  20241108    코스피    2561.15  448223481   9917837758277  2085788739364485
242  20241111    코스피    2531.66  570896056  11282657786521  2061562160237548
243  20241112    코스피    2482.57  734831995  12841318178562  2021297048796007

[244 rows x 6 columns]
243    2024년 11월 12일자 코스피 지수는? 2482.57
Name: CLSPRC_

#### ─────────────────────────────────────────────────────────────────────────────────────────

## 주식 종목 모두 확인하기
## stock_info.txt 파일로 모든 종목의 코드와 이름을 내보내기함.

### 나중에 종목명을 정확하게 골라서 입력해야할 때 사용

In [14]:
# API URL
url = "http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd" 

# 인증키 설정
auth_key = "956087495A1D4769A88B0F3411D0890EF02445EB"  # 제공받은 인증키로 대체하세요

# 헤더작성
headers = {
    "AUTH_KEY": auth_key,

}

# 요청 파라미터
params = {
    "basDd": "20241112"
}

# API 요청
response = requests.get(url, headers=headers, params=params)


# 응답 확인하기
if response.status_code == 200:
    data = response.json()  # JSON 형식으로 변환
    df_stock = pd.DataFrame(data['OutBlock_1'])[['BAS_DD', 'ISU_CD', 'ISU_NM', 'SECT_TP_NM', 'TDD_CLSPRC',
                                                 'ACC_TRDVOL', 'ACC_TRDVAL', 'MKTCAP', 'LIST_SHRS']]
    
    print(df_stock)
    
    # 주식 코드 번호(ISU_CD)와 주식명(ISU_NM)만 추출하여 새로운 데이터프레임 생성
    df_stock_info = df_stock[['ISU_CD', 'ISU_NM']]
    df_stock_info.to_csv('stock_info.txt', sep='\t', index=False, header=True, encoding='utf-8')

    print(df_stock_info)

else:
    print(f"Error {response.status_code}: {response.text}")

       BAS_DD  ISU_CD   ISU_NM SECT_TP_NM TDD_CLSPRC ACC_TRDVOL  ACC_TRDVAL  \
0    20241112  095570   AJ네트웍스          -       4575     121676   558642375   
1    20241112  006840    AK홀딩스          -      11790       6065    71568580   
2    20241112  027410      BGF          -       3550      81539   288187180   
3    20241112  282330   BGF리테일          -     107600      33614  3625818900   
4    20241112  138930  BNK금융지주          -       9490     891579  8562053990   
..        ...     ...      ...        ...        ...        ...         ...   
954  20241112  079980      휴비스          -       2425     144586   351428125   
955  20241112  005010      휴스틸          -       4060     602511  2509360060   
956  20241112  000540     흥국화재          -       3240      75203   247148950   
957  20241112  000545    흥국화재우          -       4775       4204    20071005   
958  20241112  003280     흥아해운          -       1801    1299214  2344234761   

            MKTCAP  LIST_SHRS  
0     207031372425 

#### ─────────────────────────────────────────────────────────────────────────────────────────

## 특정 종목을 선택하여 진행하기 (예시 : 삼성전자)
## 특정 종목의 종가를 확인하고, 베타값을 구하기

In [15]:
stock_name = '삼성전자'
#특정 종목의 주식 정보 저장하기

# API URL
url = "http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd" 

# 인증키 설정
auth_key = "956087495A1D4769A88B0F3411D0890EF02445EB"  # 제공받은 인증키로 대체하세요

# 헤더작성
headers = {
    "AUTH_KEY": auth_key,

}

df_stock_1 = pd.DataFrame()

current_date = start_date
while current_date <= end_date:
    bas_d = current_date.strftime("%Y%m%d")
    
    # 요청 파라미터 설정
    params = {
        "basDd": bas_d,
    }
    
    # API 요청
    response = requests.get(url, headers=headers, params=params)
    
    # 응답 확인 및 데이터 처리
    if response.status_code == 200:
        data = response.json()  # 응답 JSON 형식으로 파싱
        if 'OutBlock_1' in data and data['OutBlock_1']:
            df_stock = pd.DataFrame(data['OutBlock_1'])[['BAS_DD', 'ISU_CD', 'ISU_NM', 'SECT_TP_NM', 'TDD_CLSPRC',
                                                        'ACC_TRDVOL', 'ACC_TRDVAL', 'MKTCAP', 'LIST_SHRS']]
            df2 = df_stock[df_stock['ISU_NM'] == stock_name]

            df_stock_1 = pd.concat([df_stock_1, df2], ignore_index=True)
    else:
        print(f"Error {response.status_code}: {response.text}")

    # 다음 날짜로 이동
    current_date += timedelta(days=1)
    
print(df_stock_1)

# 2024년 11월 12일자 특정 주식의 주가를 출력한다면?
print(f'2024년 11월 12일자 {stock_name} 지수는? '+kospi_df[kospi_df['BAS_DD'] == '20241112']['CLSPRC_IDX'])

       BAS_DD  ISU_CD ISU_NM SECT_TP_NM TDD_CLSPRC ACC_TRDVOL     ACC_TRDVAL  \
0    20231113  005930   삼성전자          -      70400    9246919   653603022224   
1    20231114  005930   삼성전자          -      70800    9567984   678176565400   
2    20231115  005930   삼성전자          -      72200   20148677  1449276387300   
3    20231116  005930   삼성전자          -      72800   15860451  1151617384100   
4    20231117  005930   삼성전자          -      72500   11494644   834898662931   
..        ...     ...    ...        ...        ...        ...            ...   
239  20241106  005930   삼성전자          -      57300   22092218  1262148460900   
240  20241107  005930   삼성전자          -      57500   17043102   982114998400   
241  20241108  005930   삼성전자          -      57000   13877396   799664427482   
242  20241111  005930   삼성전자          -      55000   29811326  1654820869900   
243  20241112  005930   삼성전자          -      53000   37962881  2037790866499   

              MKTCAP   LIST_SHRS  
0   

## 베타값 구하는 함수 정의 (혜진님 코드에서 조금 변경)

In [16]:
# 개별 주식과 코스피 지수의 월별 수익률을 불러오는 함수
def get_monthly_returns(stock_data, kospi_data):
    try:
        # 코스피 지수 데이터 타입 변환 (원본에 영향이 가지 않도록 복사본 사용)
        kospi_data = kospi_data.copy()
        kospi_data['종가'] = kospi_data['종가'].astype(float)

        # 코스피 지수 데이터 처리
        kospi_data = kospi_data.resample('MS').first()  # 매월 첫 번째 날의 데이터
        kospi_data['KOSPI Return'] = kospi_data['종가'].pct_change()

        # 개별 주식 데이터 타입 변환 (원본에 영향이 가지 않도록 복사본 사용)
        stock_data = stock_data.copy()
        stock_data['종가'] = stock_data['종가'].astype(float)

        # 개별 주식 데이터 처리
        stock_data = stock_data.resample('MS').first()  # 매월 첫 번째 날의 데이터
        stock_data['Stock Return'] = stock_data['종가'].pct_change()

        # 결측값 제거
        kospi_data.dropna(inplace=True)
        stock_data.dropna(inplace=True)

        # 수익률 데이터를 합치기
        data = pd.DataFrame({
            'KOSPI': kospi_data['KOSPI Return'],
            'Stock': stock_data['Stock Return']
        }).dropna()

        return data

    except Exception as e:
        print(f"데이터 처리 중 오류 발생: {e}")
        return None

# 베타 계산 함수
def calculate_beta(data):
    if data is None or data.empty:
        return None

    try:
        X = data['KOSPI'].values.reshape(-1, 1)
        y = data['Stock'].values.reshape(-1, 1)

        # 회귀 모델 생성 및 훈련
        model = LinearRegression()
        model.fit(X, y)

        # 베타 값 추출
        beta = model.coef_[0][0]
        return beta
    except Exception as e:
        print(f"베타 계산 중 오류 발생: {e}")
        return None

In [17]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# 베타 계산을 위한 데이터 준비
kospi_data = kospi_df[['BAS_DD', 'CLSPRC_IDX']].copy()
kospi_data.rename(columns={'BAS_DD': '날짜', 'CLSPRC_IDX': '종가'}, inplace=True)
kospi_data['날짜'] = pd.to_datetime(kospi_data['날짜'], format='%Y%m%d')
kospi_data.set_index('날짜', inplace=True)

stock_data = df_stock_1[['BAS_DD', 'TDD_CLSPRC']].copy()
stock_data.rename(columns={'BAS_DD': '날짜', 'TDD_CLSPRC': '종가'}, inplace=True)
stock_data['날짜'] = pd.to_datetime(stock_data['날짜'], format='%Y%m%d')
stock_data.set_index('날짜', inplace=True)

# 베타 계산
returns_data = get_monthly_returns(stock_data, kospi_data)
if returns_data is not None and not returns_data.empty:
    beta = calculate_beta(returns_data)
    if beta is not None:
        print(f"{stock_name}의 베타 값: {beta}")
    else:
        print("베타 계산 실패.")
else:
    print("수익률 데이터를 로드하는 데 실패했습니다.")


삼성전자의 베타 값: 1.8554505218353483


#### ─────────────────────────────────────────────────────────────────────────────────────────

### dart_api 로 제무제표 데이터셋 불러오기 

In [18]:
dart_api_key = '0728a6d81b70b86763fd38daafcc1fccff0beedb'

# 기업 고유번호 XML 파일 다운로드 URL
url = f'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={dart_api_key}'

# XML 파일 다운로드
response = requests.get(url)
zip_path = 'corpCode.zip'

with open(zip_path, 'wb') as file:
    file.write(response.content)

# ZIP 파일 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

# XML 파일 파싱
xml_file = 'CORPCODE.xml'
tree = ET.parse(xml_file)
root = tree.getroot()

# 삼성전자의 고유번호 찾기
corp_name = stock_name
corp_code = None

for company in root.findall('list'):
    if company.find('corp_name').text == corp_name:
        corp_code = company.find('corp_code').text
        break

if corp_code:
    print(f"{stock_name}의 고유번호: {corp_code}")
else:
    print("{stock_name}를 찾을 수 없습니다.")

# 사용이 끝난 파일 삭제
os.remove(zip_path)
os.remove(xml_file)


삼성전자의 고유번호: 00126380


In [19]:
# 지표 사용 관련 개발가이드 https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2022001

# 보고서코드 : 1분기  // 11013
# 보고서코드 : 반기   // 11012
# 보고서코드 : 3분기  // 11014
# 보고서코드 : 사업보고서 // 11011

# 수익성지표 : M210000 안정성지표 : M220000 성장성지표 : M230000 활동성지표 : M240000


url = 'https://opendart.fss.or.kr/api/fnlttSinglIndx.json'

# 요청 파라미터 설정
params = {
    'crtfc_key': dart_api_key,
    'corp_code': corp_code,
    'bsns_year': '2024',
    'reprt_code': '11013',  # 1분기 보고서 코드
    'idx_cl_code': 'M210000'  # 수익성지표 (CFS)
}

# API 요청
response = requests.get(url, params=params)
data = response.json()

# 응답 상태 확인
if data.get('status') == '000':  # 성공 상태 코드
    index_class = pd.DataFrame(data['list'])
else:
    print("API 오류:", data.get('message'))

index_class

,reprt_code,bsns_year,corp_code,stock_code,stlm_dt,idx_cl_code,idx_cl_nm,idx_code,idx_nm,idx_val
0,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211100,세전계속사업이익률,9.393
1,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211200,순이익률,9.393
2,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211250,총포괄이익률,16.338
3,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211300,매출총이익률,36.194
4,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211400,매출원가율,63.806
5,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211550,ROE,1.837
6,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211800,판관비율,27.008
7,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M212000,총자산영업이익률,1.426
8,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M212100,총자산세전계속사업이익률,1.458
9,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M212200,자기자본영업이익률,1.796
